# Project Luther - Analysis

In [2]:
import numpy as np
import pandas as pd

import dateutil.parser
import datetime
from datetime import timedelta
import math
import calendar

import statsmodels.api as sm
import statsmodels
import seaborn as sns
from numpy import linalg
import patsy

import pandas as pd
from pymongo import MongoClient

from statsmodels.formula.api import ols

import matplotlib.pyplot as plt

%matplotlib inline

## Pull in scraped data

The data from our scraping resides in a Mongo database.

Let's go get it...

In [3]:
client = MongoClient()
db = client.willco

In [4]:
df = pd.DataFrame(list(db.PropertyInfo.find()))

Pulling from Mongo orders columns alphabetically (and adds an ID column).
Reorganizing the data columns to be more in line with how it's presented on the website...

In [17]:
df = df[[
    'PIN', 'PropClass', 'Address', 'City', 'Zip', 'SaleDate', 'SaleAmt',
    'TaxRate', 'ASL', 'ASFL', 'AI', 'ASB', 'ASFB', 'ASTotal', 'ASFTotal',
    'Subdivision', 'FullBath', 'Style', 'HalfBath', 'LivingSqFt', 'CentralAir',
    'BldgSqFt', 'Fireplace', 'YearBuilt', 'Porch', 'Basement', 'Attic',
    'Garage', 'Lot', 'Block', 'Unit', 'Building', 'Area', 'Status'
]]

In [18]:
df.head()

,PIN,PropClass,Address,City,Zip,SaleDate,SaleAmt,TaxRate,ASL,ASFL,...,Porch,Basement,Attic,Garage,Lot,Block,Unit,Building,Area,Status
0,04-10-01-101-001-0000,COMMERCIAL,2800 SCHWEITZER RD,JOLIET,60421,N/A,N/A,8.5817 (2016),"41,990",0,...,NO,NONE,NO,NONE,1,N/A,N/A,N/A,N/A,N/A
1,05-06-01-101-001-0000,COMMERCIAL,1202 ESSINGTON RD,JOLIET,60435,10/19/1987,"$140,000",10.2841 (2016),"149,410",0,...,NO,???,NO,ATTACHED,N/A,N/A,N/A,N/A,N/A,N/A


## Massage the Data

The data is stored in the format or fashion as it existed on the website.  We need a number of
transformations in order to get the data where we can use it.

### Encapsulate the changes in a function

In [ ]:
def masseuse(data):
    ''' Initial field transformations to facilitate further analysis
    Input: data - the dataframe
    Changes are made in place.
    '''
    
    data['SaleDate'] = data['SaleDate'].apply(lambda x: dateutil.parser.parse(x))

    data['Longevity'] = data['SaleDate'].apply(lambda x: dateutil.parser.parse('2018-01-01') - x)

    data['Longevity']=data['Longevity'].apply(lambda x: x.total_seconds()/86400.)

    data['SaleAmt']=data['SaleAmt'].replace('[\$,]', '', regex=True).astype(float)

    data['TaxRate']=data['TaxRate'].replace('\(\d+\)', '', regex=True).astype(float)

    data['ASTotal']=data['ASTotal'].replace('[,]', '', regex=True).astype(float)

    data['ASFTotal']=data['ASFTotal'].replace('[,]', '', regex=True).astype(float)

    data['LivingSqFt']=data['LivingSqFt'].replace(' Sq. Feet', '', regex=True).astype(float)

    data['BldgSqFt']=data['BldgSqFt'].replace(' Sq. Feet', '', regex=True).astype(float)

    data['CentralAir'].replace('YES',1.,inplace=True)
    data['CentralAir'].replace('NO',0.,inplace=True)
    data['CentralAir'] = data['CentralAir'].astype(float)

    data['Fireplace'].replace('YES',1.,inplace=True)
    data['Fireplace'].replace('NO',0.,inplace=True)
    data['Fireplace'] = data['Fireplace'].astype(float)

    data['Porch'].replace('YES',1.,inplace=True)
    data['Porch'].replace('NO',0.,inplace=True)
    data['Porch'] = data['Porch'].astype(float)

### Implement the Changes

In [ ]:
masseuse()

## Prepare to Model

### Remove rows with no target data

### Select rows with data for all chosen features

For the most part, the records available from our data source either has complete information or a portion of the information is simply marked unavailable.  Therefore, when this portion of data is missing for a feature, it's likely missing for the same set of features.

### Outlier Treatment

Earlier exploration with smaller datasets showed that a number of our features have significant outliers.  Switching to log helped to capture the data better but there were still outliers.

#### Exploration

#### Decisions

### Prepare the Model

## Model Execution

Text